# Test lógico equivalencia

In [1]:
import sys

sys.path.append("../src")

In [2]:
import nltk
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm
from pyprover import *

from nltk.sem.logic import LogicParser

lp = LogicParser()

In [3]:
from config.config import PATHS
from parser.parser_utils import Parse2pyprover

In [4]:
data_folder = PATHS['training_data_folder']
data_path = Path(data_folder, 'equivalencia_5_fol.csv')
data = pd.read_csv(data_path, names=['Sentence 1', 'Sentence 2', 'Relation'], sep=";", nrows=5)
print(f'Data shape: {data.shape}')
data.tail()

Data shape: (5, 3)


,Sentence 1,Sentence 2,Relation
0,-(exists x.(ESCRITOR(x) & SALADO(x)) & all x.(...,exists x.(ESCRITOR(x) & SALADO(x) & -DESCANSAR...,1
1,-(exists x.(ESCRITOR(x) & ENOJAR(x)) & all x.(...,exists x.(ESCRITOR(x) & ENOJAR(x) & -DESCANSAR...,1
2,-(exists x.(ESCRITOR(x) & AMPLIO(x)) & all x.(...,exists x.(ESCRITOR(x) & AMPLIO(x) & -DESCANSAR...,1
3,-(exists x.(ESCRITOR(x) & JOVEN(x)) & all x.((...,exists x.(ESCRITOR(x) & JOVEN(x) & -DESCANSAR(x)),1
4,-(exists x.(ESCRITOR(x) & TORCIDO(x)) & all x....,exists x.(ESCRITOR(x) & TORCIDO(x) & -DESCANSA...,1


In [6]:
sentences1 = data['Sentence 1'].tolist()
sentences2 = data['Sentence 2'].tolist()

In [12]:
def test_equivalencia(sentence1:str, sentence2:str) -> bool:
    '''
    Test equivalence between two sentences.
    '''
    #Test sentence1 implies sentence2
    sentence1_nltk = lp.parse(f'{sentence1}')
    print(f'Sentence 1 NLTK: {sentence1_nltk}')
    sentence1_pyprover = Parse2pyprover.parse(sentence1_nltk)

    sentence2_nltk = lp.parse(f'{sentence2}')
    print(f'Sentence 2 NLTK: {sentence2_nltk}')
    sentence2_pyprover = Parse2pyprover.parse(sentence2_nltk)

    premisas = [sentence1_pyprover]
    conclusion = sentence2_pyprover
    resultado1 = proves(premisas, conclusion)
    print('Resultado:', resultado1)

    # Test sentence2 implies sentence1

    premisas = [sentence2_pyprover]
    conclusion = sentence1_pyprover
    resultado2 = proves(premisas, conclusion)
    print('Resultado:', resultado2)

    return resultado1 and resultado2

In [13]:
tested = list()
for index, row in data.iterrows():
    sentences1 = row['Sentence 1']
    sentences2 = row['Sentence 2']
    print('Sentences:', sentences1, sentences2)
    relation = row['Relation']
    result = test_equivalencia(sentences1, sentences2)
    tested.append(int(result))

data['Tested'] = tested
data.head()

Sentences: -(exists x.(ESCRITOR(x) & SALADO(x)) & all x.((ESCRITOR(x) & SALADO(x)) -> DESCANSAR(x))) exists x.(ESCRITOR(x) & SALADO(x) & -DESCANSAR(x))
Sentence 1 NLTK: -(exists x.(ESCRITOR(x) & SALADO(x)) & all x.((ESCRITOR(x) & SALADO(x)) -> DESCANSAR(x)))
Sentence 2 NLTK: exists x.(ESCRITOR(x) & SALADO(x) & -DESCANSAR(x))


KeyboardInterrupt: 

In [69]:
data[data['Tested'] != data['Relation']]

,Sentence 1,Sentence 2,Relation,Tested


In [70]:
data.tail()

,Sentence 1,Sentence 2,Relation,Tested
175,(exists x.JINETE(x) & all x.(JINETE(x) -> (ENT...,exists x.(JINETE(x) & -ENTRAR(x) & -DEAMBULAR(x)),1,1
176,(exists x.JINETE(x) & all x.(JINETE(x) -> (ACO...,exists x.(JINETE(x) & -ACORDAR(x) & -ENTRAR(x)),1,1
177,(exists x.JINETE(x) & all x.(JINETE(x) -> (ACO...,exists x.(JINETE(x) & -ACORDAR(x) & -DEAMBULAR...,1,1
178,(exists x.JINETE(x) & all x.(JINETE(x) -> (DEA...,exists x.(JINETE(x) & -DEAMBULAR(x) & -ENTRAR(x)),1,1
179,(exists x.JINETE(x) & all x.(JINETE(x) -> (DEA...,exists x.(JINETE(x) & -DEAMBULAR(x) & -ACORDAR...,1,1
